In [1]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import json
import pickle

# Load the Bi_LSTM
model = load_model('Bi_LSTM_ED.h5')

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Load max lengths
with open('lengths.json', 'r') as f:
    lengths = json.load(f)

max_length_claims = lengths['max_length_claims']
max_length_evidence = lengths['max_length_evidence']

In [4]:
# Load trial data
trial_data = pd.read_csv('test.csv')
trial_data['Claim'] = trial_data['Claim'].astype(str)
trial_data['Evidence'] = trial_data['Evidence'].astype(str)

# Tokenization
trial_claims_seq = tokenizer.texts_to_sequences(trial_data['Claim'].tolist())
trial_evidence_seq = tokenizer.texts_to_sequences(trial_data['Evidence'].tolist())

# Pad the sequences to the max length
trial_claims_pad = pad_sequences(trial_claims_seq, maxlen=28)
trial_evidence_pad = pad_sequences(trial_evidence_seq, maxlen=273)

# Predict the labels
predictions = model.predict([trial_claims_pad, trial_evidence_pad])
predictions_trial = (predictions > 0.5).astype(int)

# Display the shape of the predictions
print(predictions_trial.shape)

# Save the predictions to an Excel file
prediction_df = pd.DataFrame(predictions_trial, columns=['prediction'])
prediction_df.to_csv('output.csv', mode='w', index=False)

147/147 [==============================] - 6s 17ms/step
(4691, 1)
